In [9]:
import cherrypy
import json
import redis
import uuid
import requests
from redis.commands.json.path import Path

In [10]:
publicid = "redis-15072.c77.eu-west-1-1.ec2.cloud.redislabs.com"
username = "default"
password = "53R8YAlL81zAHIEVcPjwjzcnVQoSPhzt"
redis_port = 15072

redis_client = redis.Redis(host = publicid, password = password, username = username, port = redis_port)
print("Is connected? ", redis_client.ping())

Is connected?  True


In [11]:
class Device(object):
    exposed = True
    
    def GET(self):
        # response = {'mac_address':[]}
        items = []
        keys = redis_client.keys("*:battery")
        print(len(keys))
        for key in keys:
            key = key.decode()
            #subitem = redis_client.get(key)
            #ma = key.removeprefix("")
            #item[f"{key.split(':')[0]}"] = subitem
            items.append(key.split(":")[0])

        response = json.dumps(items)
        return response


class SpecificDevice:
    exposed = True

    def GET(self, *path, **query):
        pass


In [12]:
if __name__=='__main__':
    conf = {'/': {'request.dispatch': cherrypy.dispatch.MethodDispatcher(), 'tools.sessions.on':True}}
    cherrypy.tree.mount(Device(), '/devices', conf)
    cherrypy.config.update({'server.socket_host': '0.0.0.0'})
    cherrypy.config.update({'server.socket_port': 8080})
    cherrypy.engine.start()
    cherrypy.engine.block()

[16/Jan/2023:11:59:30] ENGINE Bus STARTING
[16/Jan/2023:11:59:30] ENGINE Started monitor thread 'Autoreloader'.
[16/Jan/2023:11:59:30] ENGINE Serving on http://0.0.0.0:8080
[16/Jan/2023:11:59:30] ENGINE Bus STARTED


7
127.0.0.1 - - [16/Jan/2023:11:59:33] "GET /devices HTTP/1.1" 200 118 "" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
